# Camera Calibration Notebook

In [4]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import os

In [21]:
dirs = ["./field_checkerboard_video/"]

In [ ]:
vv_s = 0.08684819327608279
vv_n = 0.06564555425303907
vv_i = 0.04630976610304423
vv_f = 0.06755149834582377

vs_s = 0.04043636146989359
vs_n = 0.0397322715506765
vs_i = 0.04385700211221671
vs_f = 0.043480910391920216

## Pixel Reprojection error ( Total and Across Bins)

In [ ]:
if not (os.path.exists("./vid_hard_check/")):
    os.makedirs("./vid_hard_check/")

In [82]:
global_dict_no_eq = {}
for d in dirs:
    flist = sorted(glob.glob(os.path.join(d, "*.mp4")))
    for vf in flist:
        print(vf)
        vidcap = cv2.VideoCapture(vf)

        files = glob.glob("./vid_hard_check/*")
        for fl in files:
            os.remove(fl)

        success,image = vidcap.read()
        count = 0
        success = True
        while success:
            success,image = vidcap.read()
            cv2.imwrite("./vid_hard_check/frame%d.png" % count, image)     # save frame as PNG file
            if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                break
            count += 1

        print(count)

        nx = 5
        ny = 5
        bins = 6
        
        
        interval = count/bins

        mesh = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
        objp = np.hstack((mesh, np.zeros(shape = (nx*ny,1))))
        objp = np.array(objp, dtype = np.float32)

        fname = glob.glob('./vid_hard_check/*.png')
        
        for j in range(1,4):

            imgpoints = []
            objpoints = []
            imgpoints2_list = []
            detected_list = []
            bin_error = []
            bincount = np.zeros(6)
            fx = j*0.1
            fy = j*0.1
            for i,f in enumerate(fname):
                try:
                    imcv = cv2.imread(f)
                    imcv = cv2.cvtColor(imcv, cv2.COLOR_BGR2GRAY)

####### Uncomment below two lines to apply histogram equalization for removing shiny effects, cl = 2, 5 or 10 work best #####                    

#                     cli = cv2.createCLAHE(clipLimit = cl)
#                     imcv = cli.apply(imcv)

                    imcv = cv2.GaussianBlur(imcv, ksize = (5,5), sigmaX = 1, sigmaY = 1)
                    imcv = cv2.resize(imcv, (0, 0), fx=fx, fy=fy, interpolation=cv2.INTER_AREA)


                    retval, corners = cv2.findChessboardCorners(imcv,(nx,ny), None)
                    if (retval == True):

                        detected_list.append(f)
###### Uncomment below lines to plot the detected checkerboard corners #######

#                         plt.figure()
#                         cv2.drawChessboardCorners(imcv, (nx, ny), corners, retval)
#                         plt.imshow(imcv, cmap='gray')

                        imgpoints.append(corners)
                        objpoints.append(objp*20)
                        bincount[int(i/interval)] += 1
                              
                except Exception as e:
                    pass

            print(fx, len(objpoints))
            mean_error = 1.0
            mtx = dist = rvecs = tvecs = []
            if (len(objpoints) != 0):
                retval, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, imcv.shape[::-1], None, None)
                mean_error = 0.0
                
                tot_len = len(objpoints)
                st = 0
                for b in range(bins):
                    berror = 0.0
                    if b != 0 :
                        st += bincount[b-1]
                    else: 
                        st = 0
                    end = st + bincount[b]
                    for i in range(int(st),int(end)):
                        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
                        imgpoints2_list.append(imgpoints2)
                        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
                        berror += error
                    if bincount[b] != 0:
                        bin_error.append(berror/bincount[b])
                    else:
                        bin_error = 1.0

                for i in range(len(objpoints)):
                    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
                    imgpoints2_list.append(imgpoints2)
                    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
                    mean_error += error
                print( "total error: {} for scaling of {} with {} number of images for file {}".format(mean_error/len(objpoints), fx, len(objpoints), vf))

            if vf not in global_dict_no_eq:
                global_dict_no_eq[vf] = []

            global_dict_no_eq[vf].append([fx, len(objpoints), mean_error/(len(objpoints)+0.00001), mtx, dist,detected_list,imgpoints, bin_error, bincount])

./180/c1_180.mp4
512
0.1 32
total error: 0.045188109597296015 for scaling of 0.1 with 32 number of images for file ./180/c1_180.mp4
0.2 131
total error: 0.0743747954100561 for scaling of 0.2 with 131 number of images for file ./180/c1_180.mp4
0.30000000000000004 169
total error: 0.14461012825005387 for scaling of 0.30000000000000004 with 169 number of images for file ./180/c1_180.mp4
./180/c3_180.mp4
518
0.1 37
total error: 0.06838695833178102 for scaling of 0.1 with 37 number of images for file ./180/c3_180.mp4
0.2 115
total error: 0.12285693125425 for scaling of 0.2 with 115 number of images for file ./180/c3_180.mp4
0.30000000000000004 123
total error: 0.1945220697507891 for scaling of 0.30000000000000004 with 123 number of images for file ./180/c3_180.mp4
./180/c2_180.mp4
493
0.1 26
total error: 0.07176843124042381 for scaling of 0.1 with 26 number of images for file ./180/c2_180.mp4
0.2 92
total error: 0.15528006330739455 for scaling of 0.2 with 92 number of images for file ./180/

## Plot of reprojection errors across bins

In [83]:
import pickle
pickle_out = open("180_4.pickle","wb")
pickle.dump(global_dict_no_eq, pickle_out)
pickle_out.close()

In [84]:
plot_dict1 = {}
plot_dict2 = {}
plot_dict3 = {}


bin_dict1 = {}
bin_dict2 = {}
bin_dict3 = {}


In [86]:
for key in global_dict_no_eq:
    l = global_dict_no_eq[key]
    plot_dict1[key] = l[0][7]
    bin_dict1[key] = l[0][8]
    plot_dict2[key] = l[1][7]
    bin_dict2[key] = l[1][8]
    plot_dict3[key] = l[2][7]
    bin_dict3[key] = l[2][8]

In [87]:
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets

py.offline.init_notebook_mode(connected=True)

In [88]:
layout1 = go.Layout(
    title='Reprojection Error Plot(fx = 0.1)',
    yaxis = dict(title='error'),
    xaxis = dict(title='bins')
)

trace11 = go.Scatter(
    x = np.arange(6),
    y = plot_dict1['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace12 = go.Scatter(
    x = np.arange(6),
    y = plot_dict1['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace13 = go.Scatter(
    x = np.arange(6),
    y = plot_dict1['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace14 = go.Scatter(
    x = np.arange(6),
    y = plot_dict1['./180/c4_180.mp4'],
    mode='lines+markers'
)


layout2 = go.Layout(
    title='Reprojection Error Plot(fx = 0.2)',
    yaxis = dict(title='error'),
    xaxis = dict(title='bins')
)

trace21 = go.Scatter(
    x = np.arange(6),
    y = plot_dict2['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace22 = go.Scatter(
    x = np.arange(6),
    y = plot_dict2['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace23 = go.Scatter(
    x = np.arange(6),
    y = plot_dict2['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace24 = go.Scatter(
    x = np.arange(6),
    y = plot_dict2['./180/c4_180.mp4'],
    mode='lines+markers'
)


layout3 = go.Layout(
    title='Reprojection Error Plot(fx = 0.3)',
    yaxis = dict(title='error'),
    xaxis = dict(title='bins')
)

trace31 = go.Scatter(
    x = np.arange(6),
    y = plot_dict3['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace32 = go.Scatter(
    x = np.arange(6),
    y = plot_dict3['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace33 = go.Scatter(
    x = np.arange(6),
    y = plot_dict3['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace34 = go.Scatter(
    x = np.arange(6),
    y = plot_dict3['./180/c4_180.mp4'],
    mode='lines+markers'
)

In [89]:
fig1 = go.Figure(data=[trace11,trace12,trace13,trace14], layout=layout1)
fig2 = go.Figure(data=[trace21,trace22,trace23,trace24], layout=layout2)
fig3 = go.Figure(data=[trace31,trace32,trace33,trace34], layout=layout3)

In [90]:
py.offline.iplot(fig1)
py.offline.iplot(fig2)
py.offline.iplot(fig3)

In [91]:
layout1 = go.Layout(
    title='Images detected Plot(fx = 0.1)',
    yaxis = dict(title='num_image'),
    xaxis = dict(title='bins')
)

trace11 = go.Scatter(
    x = np.arange(6),
    y = bin_dict1['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace12 = go.Scatter(
    x = np.arange(6),
    y = bin_dict1['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace13 = go.Scatter(
    x = np.arange(6),
    y = bin_dict1['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace14 = go.Scatter(
    x = np.arange(6),
    y = bin_dict1['./180/c4_180.mp4'],
    mode='lines+markers'
)


layout2 = go.Layout(
    title='Images detected Plot(fx = 0.2)',
    yaxis = dict(title='num_image'),
    xaxis = dict(title='bins')
)

trace21 = go.Scatter(
    x = np.arange(6),
    y = bin_dict2['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace22 = go.Scatter(
    x = np.arange(6),
    y = bin_dict2['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace23 = go.Scatter(
    x = np.arange(6),
    y = bin_dict2['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace24 = go.Scatter(
    x = np.arange(6),
    y = bin_dict2['./180/c4_180.mp4'],
    mode='lines+markers'
)


layout3 = go.Layout(
    title='Images detected Plot(fx = 0.3)',
    yaxis = dict(title='num_image'),
    xaxis = dict(title='bins')
)

trace31 = go.Scatter(
    x = np.arange(6),
    y = bin_dict3['./180/c1_180.mp4'],
    mode='lines+markers'
)

trace32 = go.Scatter(
    x = np.arange(6),
    y = bin_dict3['./180/c2_180.mp4'],
    mode='lines+markers'
)

trace33 = go.Scatter(
    x = np.arange(6),
    y = bin_dict3['./180/c3_180.mp4'],
    mode='lines+markers'
)

trace34 = go.Scatter(
    x = np.arange(6),
    y = bin_dict3['./180/c4_180.mp4'],
    mode='lines+markers'
)

fig1 = go.Figure(data=[trace11,trace12,trace13,trace14], layout=layout1)
fig2 = go.Figure(data=[trace21,trace22,trace23,trace24], layout=layout2)
fig3 = go.Figure(data=[trace31,trace32,trace33,trace34], layout=layout3)

py.offline.iplot(fig1)
py.offline.iplot(fig2)
py.offline.iplot(fig3)

## Tester for checkerboard detection

In [ ]:
for j in range(1,11):
    im = cv2.imread("./with_lower_reflectivity.png")
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im[im[:,:] >= 128] = 255
    im[im[:,:] < 128] = 0
    im = 255 - im
    plt.figure()
    plt.imshow(im, cmap='gray')
    fx = j*0.1
    fy = j*0.1
    im = cv2.GaussianBlur(im, ksize = (5,5), sigmaX = 1, sigmaY = 1)
    im = cv2.resize(im, (0, 0), fx=fx, fy=fy, interpolation=cv2.INTER_AREA)
    retval, corners = cv2.findChessboardCorners(im,(nx,ny), None)
    print (retval, fx)
    if retval == True:
        plt.figure()
        cv2.drawChessboardCorners(im, (nx, ny), corners, retval)
        plt.imshow(im, cmap = 'gray')